In [32]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import requests

In [112]:
def convert_num_nam(string):
    "Given a string that is a concatenation of a jersey number and the name, return the two substring seperate"

    num = [letter for letter in string if letter.isdigit()]
    num = "".join(map(str, num))
    nam = [letter for letter in string if not letter.isdigit()]
    nam = "".join(map(str, nam))

    return num, nam
    
def get_field_df(soup):
    field_ratings = []
    field_players = []
    field_numbers = []
    for field_player in soup.find_all(class_=re.compile("LineupPlayerContainer")):
        rating_element = field_player.find_all(class_=re.compile("PlayerRatingStyled"))
        if len(rating_element) == 0:
            field_ratings.append("nan")
        else:
            field_ratings.append(rating_element[0].find_all("span")[0].get_text())
        
        player_text = field_player.find(class_=re.compile("LineupPlayerText"))
        num, name = convert_num_nam(player_text.get_text())
        field_players.append(name)
        field_numbers.append(num)

    return pd.DataFrame({"number": field_numbers, "name": field_players, "rating": field_ratings})
    
def get_bench_df(soup):
    bench_ratings = []
    bench_players = []
    bench_numbers = []
    for subsoup in soup.find_all(class_=re.compile("LeftBenchItem ")):  # the space is very important in the regex
        if 'PlayerRating' in str(subsoup):
            # extract name and number
            num, name = convert_num_nam((subsoup.find_all('span')[-2].get_text()))
            bench_players.append(name)
            bench_numbers.append(num)

            # get rating
            rating_element = subsoup.find(class_=re.compile("PlayerRatingStyled"))
            bench_ratings.append(rating_element.find('span').get_text())

    return pd.DataFrame({"number": bench_numbers, "name": bench_players, "rating": bench_ratings})

In [115]:
page = requests.get("https://www.fotmob.com/match/3602683/matchfacts/fc-utrecht-vs-go-ahead-eagles")
soup = BeautifulSoup(page.content, 'html.parser')
pd.concat([get_field_df(soup), get_bench_df(soup)]).reset_index(drop=True)

,number,name,rating
0,1,Paes,nan
1,20,Zagre,6.4
2,2,van der Maarel,7.9
3,33,van der Hoorn,7.5
4,5,ter Avest,7.4
5,27,Timber,8.0
6,6,Maher,7.8
7,30,Balk,6.4
8,23,Ramselaar,7.0
9,10,Gustafson,6.8
